In [26]:
# -------------------------------------------------------------
# nome_completo_aluno_atividade_1.ipynb
# Atividade 1 - Disciplina de Python e Análise de Dados
# -------------------------------------------------------------

# ================================
# 1. Importação de bibliotecas
# ================================
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Configuração de estilo dos gráficos
plt.style.use("seaborn-v0_8")

base_url = "https://legis.senado.gov.br/dadosabertos/"


## Siglas do Processo

In [ ]:
url_siglas = base_url + "processo/siglas"

response = requests.get(url_siglas)

# Verificando resposta
if (response.status_code == 200):
    siglas_processo = pd.DataFrame(response.json())
    siglas_processo = siglas_processo[pd.isnull(siglas_processo["dataFim"])]
else:
    print("Tratar retorno diferente de 200")

siglas_processo

,sigla,descricao,dataInicio,dataFim
1,ACL,Ato Conjunto Legislativo,2023-01-01T00:00:00,None
4,ADV,Advertencia,1946-01-01T00:00:00,None
6,AFS,Afastamento do Exercício do Mandato,2008-01-01T00:00:00,None
7,AJECN,Ajuste de emenda orçamentária,2015-01-01T00:00:00,None
9,AP.,Anteprojeto,1946-01-01T00:00:00,None
...,...,...,...,...
165,RRC,Recomendação do Conselho de Comunicação Social,2004-08-04T00:00:00,None
176,SUBEMENDA,Subemenda,1946-01-01T00:00:00,None
177,SUG,Sugestão,1900-01-01T00:00:00,None
181,TVR,Ato de Concessão e Renovação de Concessão de E...,1948-01-01T00:00:00,None


In [4]:
# ================================
# 2. Coleta de dados da API
# ================================
# Endpoint do Senado: proposições legislativas (matérias)
# Doc: https://www12.senado.leg.br/dados-abertos
url = "https://legis.senado.gov.br/dadosabertos/processo?sigla=PL&v=1"

# Exemplo: buscando proposições do ano de 2024 (pode ser alterado)
params = {"ano": 2024}
response = requests.get(url, params=params)

# Verificando resposta
print(response.status_code)


200


In [ ]:
# ================================
# 3. Transformação em DataFrame
# ================================
# A API retorna XML/JSON dependendo da rota; 
# aqui usamos JSON (quando disponível) ou convertemos.

# Obs.: para XML seria preciso usar xml.etree.ElementTree ou lxml

data = response.json()
materias = data['ListaMaterias']['Materias']['Materia']

# Criando DataFrame
df = pd.DataFrame(materias)

# Exibir primeiras linhas
df.head()


In [ ]:
# ================================
# 4. Limpeza e preparação
# ================================
# Selecionando variáveis de interesse
colunas = ["idMateria", "siglaCasa", "siglaTipo", "numero", "ano", "ementa", "descricaoSituacao"]
df = df[colunas]

# Ajustar tipos de dados
df["ano"] = df["ano"].astype(int)
df["numero"] = df["numero"].astype(int)

df.head()


In [ ]:
# ================================
# 5. Estatísticas descritivas
# ================================
print("Total de proposições:", len(df))
print("\nResumo estatístico:")
print(df.describe(include="all"))


In [ ]:
# ================================
# 6. Visualizações iniciais
# ================================

# Quantidade de proposições por tipo
df["siglaTipo"].value_counts().plot(kind="bar", figsize=(8,4))
plt.title("Distribuição por Tipo de Proposição")
plt.xlabel("Tipo")
plt.ylabel("Quantidade")
plt.show()

# Proposições por ano
df["ano"].value_counts().sort_index().plot(kind="line", marker="o", figsize=(8,4))
plt.title("Proposições por Ano")
plt.xlabel("Ano")
plt.ylabel("Quantidade")
plt.show()


In [ ]:
# ================================
# 7. Discussão preliminar (anotações)
# ================================
# - Identificar variáveis com dados ausentes.
print(df.isna().sum())

# - Refletir sobre possíveis problemas:
#   * Situações pouco padronizadas.
#   * Textos longos em 'ementa' -> análise qualitativa ou NLP.
#   * Discrepâncias entre anos com muitas/ poucas proposições.
